# Run CP2K Tests
Run the tests with different parameterizations, save the results to ASE for testing later

In [1]:
from examol.simulate.ase import ASESimulator
from examol.simulate.ase.utils import read_from_string, buffer_cell, make_ephemeral_calculator
from ase.calculators.cp2k import CP2K
from ase.db import connect
import pandas as pd
import numpy as np
import json

In [2]:
db_path = 'solv-calculations.db'
config_name = 'cp2k_blyp_dzvp'
charge = 0
solv_name = 'acn'

## Load in Reference Data
We are going to use a subset from a Gaussian dataset

In [3]:
data = pd.read_csv('reference-data.csv.gz')

In [4]:
data = data.sample(32, random_state=1).sort_values('n_atom')

## Get settings without solvents
We are going to add some options to it and use them to drive CP2K

In [5]:
sim = ASESimulator(cp2k_command='/home/lward/Software/cp2k-2022.2/exe/local/cp2k_shell.ssmp')

In [6]:
conf = sim.create_configuration(config_name, charge=0, solvent=None)
print(conf['kwargs']['inp'])

&FORCE_EVAL
&DFT
  &XC
     &XC_FUNCTIONAL BLYP
     &END XC_FUNCTIONAL
  &END XC
  &POISSON
     PERIODIC NONE
     PSOLVER MT
  &END POISSON
  &SCF
    &OUTER_SCF
     MAX_SCF 9
    &END OUTER_SCF
    &OT T
      PRECONDITIONER FULL_ALL
    &END OT
  &END SCF
&END DFT
&SUBSYS
  &TOPOLOGY
    &CENTER_COORDINATES
    &END
  &END
&END FORCE_EVAL



We want to introdroduce the SCCS settings into the "DFT" block. So, we'll adjust find/replace for convenience

In [7]:
def insert_sccs(inp: str, e0, gamma, alpha, beta) -> str:
    """Insert the SCCS settings into our input file
    
    Args:
        inp: The input block to be editted
        ..., the 6 different parameter values
    Returns:
        A new Input block
    """
    
    return inp.replace('&END SCF\n', f"""&END SCF
  &SCCS 
    ALPHA {alpha}
    BETA {beta}
    GAMMA {gamma}
    RELATIVE_PERMITTIVITY {e0}
    DERIVATIVE_METHOD CD3
    METHOD ANDREUSSI
  &END SCCS\n""")

In [8]:
print(insert_sccs(conf['kwargs']['inp'], 1, 2, 3, 4))

&FORCE_EVAL
&DFT
  &XC
     &XC_FUNCTIONAL BLYP
     &END XC_FUNCTIONAL
  &END XC
  &POISSON
     PERIODIC NONE
     PSOLVER MT
  &END POISSON
  &SCF
    &OUTER_SCF
     MAX_SCF 9
    &END OUTER_SCF
    &OT T
      PRECONDITIONER FULL_ALL
    &END OT
  &END SCF
  &SCCS 
    ALPHA 3
    BETA 4
    GAMMA 2
    RELATIVE_PERMITTIVITY 1
    DERIVATIVE_METHOD CD3
    METHOD ANDREUSSI
  &END SCCS
&END DFT
&SUBSYS
  &TOPOLOGY
    &CENTER_COORDINATES
    &END
  &END
&END FORCE_EVAL



## Define the range of parameters to test
Use &epsilon;<sub>0</sub> and &gamma; from experiment, the defaults for &rho;, and start with the &alpha; and &beta; ranges used in [the original paper](https://aip.scitation.org/doi/full/10.1063/1.3676407#_i15)

In [9]:
gamma = 29.4500  # http://www.ddbst.com/en/EED/PCP/SFT_C3.php

In [10]:
e_0 = 37.5  # https://depts.washington.edu/eooptic/linkfiles/dielectric_chart%5B1%5D.pdf

The values of &alpha; are designed such that $\alpha+\gamma$ are between 0 - 20 mN/m.

In [11]:
alphas = np.linspace(0, 20, 2) - gamma

Betas should vary between -0.1 and 0.1 GPa

In [12]:
betas = np.linspace(-0.10, 0.10, 2)

## Loop over molecules
We need to compute an energy of the structure in vacuum and in solvent for each parameter. We'll save all those results in a database so we can avoid-recomputing

In [ ]:
for xyz, smiles in zip(data['xyz'], data['smiles_1']):
    # Parse the structure and put a buffer around it
    atoms = read_from_string(xyz, 'xyz')
    buffer_cell(atoms)
    
    # Get all of the calculations run with this SMILES string
    with connect(db_path) as db:
        already_ran = [x.key_value_pairs for x in db.select(smiles=smiles)]
    
    # Run a calculation in vacuum, if not already done
    my_record = {
        'config_name': config_name,
        'smiles': smiles,
        'solvent': 'vacuum',
        'charge': charge,
    }
    if my_record not in already_ran:
        with make_ephemeral_calculator(conf) as calc:
            atoms.set_calculator(calc)
            vac_eng = atoms.get_potential_energy()

        # Save it to the database
        with connect(db_path) as db:
            db.write(atoms, **my_record)
            
    # Loop over alpha/beta options
    my_record['solvent'] = solv_name
    for beta in betas:
        for alpha in alphas:
            my_record['alpha'] = alpha
            my_record['beta'] = beta
            
            # Update the configuration
            my_conf = json.loads(json.dumps(conf))
            my_conf['kwargs']['inp'] = insert_sccs(
                conf['kwargs']['inp'], e_0, gamma, alpha, beta
            )
                                   
            # Run a calculation in vacuum if not already done
            if my_record not in already_ran:
                with make_ephemeral_calculator(my_conf) as calc:
                    atoms.set_calculator(calc)
                    atoms.get_potential_energy()

                # Save it to the database
                with connect(db_path) as db:
                    db.write(atoms, **my_record)